In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [172]:
agent_data = pd.read_csv('Data/datastorm_agent_data.csv')
policy_data = pd.read_csv("Data/datastorm_policy_data_short.csv")

In [173]:
policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [174]:
# Useful Columns
columns = ["main_holder_occupation_cd", "policy_term", "policy_payment_mode", "policy_status", "main_holder_gender", "main_holder_entry_age"]

In [175]:
snapshot_months = np.sort(pd.unique(policy_data["policy_snapshot_as_on"]))

# def divide_chunks(l, n):
#     for i in range(0, len(l), n): 
#         yield l[i:i + n]

# k = 6

# snapshot_months = list(divide_chunks(snapshot_months, k))

snapshot_months

array([20181231, 20190131, 20190228, 20190331, 20190430, 20190531,
       20190630, 20190731, 20190831, 20190901, 20191001, 20191101,
       20191201, 20200101, 20200201, 20200301, 20200401, 20200501,
       20200601, 20200701], dtype=int64)

In [176]:
valid_policy_data = policy_data[policy_data["termination_dt"].isnull()]
valid_policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [177]:
# Return cross sell status for each month
def is_cross_sell(no_of_policies):
    cross_sell = [];
    n = 6
    if(len(no_of_policies)) < 6:
        n = len(no_of_policies)

    for k in range(len(no_of_policies) - n):
        if(no_of_policies[k + n] > no_of_policies[k]):
            cross_sell.append(True)
        else:
            cross_sell.append(False)
    return cross_sell

for client_name, client_data in valid_policy_data.groupby(["client_code"]):
    no_of_policies = client_data["policy_snapshot_as_on"].value_counts().sort_index()

    if(client_name == "C100061"):
        # print(client_data.head(50))
        print(is_cross_sell(list(no_of_policies)))
        print(no_of_policies)
        
    # break
    

[False, False, False, False, True, True, True, True, True, True, False, False, False, False]
20181231    1
20190131    1
20190228    1
20190331    1
20190430    1
20190531    1
20190630    1
20190731    1
20190831    1
20190901    1
20191001    2
20191101    2
20191201    2
20200101    2
20200201    2
20200301    2
20200401    2
20200501    2
20200601    2
20200701    2
Name: policy_snapshot_as_on, dtype: int64


In [178]:
# bins = [0, 1, 5, 10, 25, 50, 100]
# df['binned'] = pd.cut(df['percentage'], bins)

In [199]:
# Nullify Terminated Users

terminated_users = policy_data[policy_data["termination_dt"].notna()].client_code.unique()

In [203]:
test_data = pd.read_csv("Data/testset.csv")

test_data["probability_of_cross_sell"] = np.random.rand(test_data.shape[0])

test_data.loc[test_data["map_client_cd"].isin(terminated_users) , "probability_of_cross_sell"] = 0

test_data


,map_client_cd,probability_of_cross_sell
0,C100003,0.406901
1,C100004,0.989584
2,C100009,0.339157
3,C100014,0.301586
4,C100015,0.536124
...,...,...
19137,C154448,0.549853
19138,C154449,0.740576
19139,C154450,0.841277
19140,C154454,0.732790


In [204]:
test_data.to_csv("Data/submission.csv")
